In [ ]:
import torch
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from model import GFR

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram

In [ ]:
def gen_dataset(params, threshold=0.5):
    with open("data/labels.pickle", "rb") as f:
        labels = pickle.load(f)
    
    chosen_ids = filter(lambda x: params[x]["evr2"] > threshold, params.keys())
    
    dataset = {}
    for cell_id in chosen_ids:
        y = labels[cell_id]
        p = params[cell_id]["params"]
        
        a = p["a"].reshape(-1)
        b = p["b"].reshape(-1)
        pc = p["g"]["poly_coeff"].reshape(-1)
        gb = p["g"]["b"].reshape(-1)
        mc = p["g"]["max_current"].reshape(-1)
        mfr = p["g"]["max_firing_rate"].reshape(-1)
        x = torch.cat([a, b, pc, gb, mc, mfr])
        
        dataset[cell_id] = (x, y, params[cell_id]["evr2"])
        
    return dataset

def get_line_name(df, cell_id):
    return df[df["specimen__id"] == cell_id]["line_name"].to_numpy()[0]

def get_labels(cell_ids):
    df = pd.read_csv("data/metadata.csv")
    line_names = ["Pvalb", "Sst", "Vip", "Htr3a", "Ndnf", "Cux2", "Nr5a1", "Ntsr1", "Rorb", "Scnn1a", "Tlx3", "Rbp4"]
    new_cell_ids = []
    labels = []
    
    for cell_id in cell_ids:
        line_name = get_line_name(df, cell_id)
        if type(line_name) != str or "|" in line_name:
            pass
        elif line_name.split("-")[0] in line_names:
            new_cell_ids.append(cell_id)
            labels.append(line_name.split("-")[0])
    
    types, counts = np.unique(labels, return_counts=True)
    types = types[counts > 20]
    new_cell_ids, labels = zip(*list(filter(lambda x: x[1] in types, zip(new_cell_ids, labels))))
    
    return np.array(new_cell_ids), np.array(labels)

def get_dataset(dataset=None, instant=False):
    if dataset is None:
        with open("data/dataset.pickle", "rb") as f:
            dataset = pickle.load(f)
    cell_ids, ys = get_labels(dataset.keys())
    #remove = []#[479298854, 489719193, 485060618, 486791945, 562535995, 321707905]# + [478110866]
    #idxs = list(filter(lambda x: cell_ids[x] in remove, range(len(cell_ids))))
    #cell_ids = [cell_ids[i] for i in range(len(cell_ids)) if i not in idxs]
    #ys = [ys[i] for i in range(len(ys)) if i not in idxs]
    xs = []
    for cell_id in cell_ids:
        x = dataset[cell_id][0].tolist()
        c0 = (x[-5]**2 - x[-3]) / x[-2]
        c1 = x[-4]**2 / x[-2]
        if not instant:
            xs.append(x[:-5] + [c0, c1, x[-1]])
        else:
            xs.append(x[8:-5] + [c0, c1, x[-1]])
    xs = np.array(xs)
    return xs, ys, cell_ids

def get_best_params_for_actv_bin_size(params, bin_size, actv_bin_size):
    best_params = {}
    
    cell_ids = set()
    for config in params:
        cell_ids = cell_ids.union(set(params[config].keys()))
    
    for cell_id in cell_ids:
        best_config = None
        best_evr = -1e10
        
        for config in params:
            if config[0] == bin_size and config[1] == actv_bin_size and cell_id in params[config] and params[config][cell_id]["evr1"] > best_evr:
                best_evr = params[config][cell_id]["evr1"]
                best_config = config
        
        # doesn't make sense
        if best_config is not None:
            best_params[cell_id] = params[best_config][cell_id]
        
    return best_params

In [ ]:
import torch.nn.functional as F
from torcheval.metrics.functional import multiclass_f1_score, multiclass_accuracy, multiclass_confusion_matrix

def summary(model, dataloader, n_classes):
    y_pred, y = get_predictions(model, dataloader, n_classes)
    f = multiclass_f1_score(y_pred, y, num_classes=n_classes, average="weighted")
    acc = multiclass_accuracy(y_pred, y, num_classes=n_classes)
    return f, acc

def get_predictions(model, dataloader, n_classes):
    ys, ys_pred = [], []
    with torch.no_grad():
        for X, y, w in dataloader:
            X = X.to(torch.float32)
            y1 = torch.einsum("ij,j->i", y, torch.arange(n_classes))
            y = y.to(torch.float32)
            ys_pred.append(model(X))
            ys.append(y1)
    return torch.cat(ys_pred), torch.cat(ys)

class ParameterDataset(torch.utils.data.Dataset):
    def __init__(self, xs, ys, label_map):
        self.n_classes = len(label_map)
        self.xs = list(map(torch.tensor, xs))
        self.ys = list(map(torch.tensor, [label_map[y] for y in ys]))
        self.ys = list(map(lambda x: F.one_hot(x, num_classes=self.n_classes), self.ys))
        
        types, counts = np.unique(ys, return_counts=True)
        ws = (1 / counts) / np.sum(1 / counts)
        w_map = {t: w for t, w in zip(types, ws)}
        self.ws = list(map(lambda x: w_map[x], ys))

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        return self.xs[idx], self.ys[idx], self.ws[idx]

from sklearn.model_selection import train_test_split

def train_model(xs, ys, pca=True, scale=True, linear=False):
    types = np.unique(ys).tolist()
    label_map = {t: i for i, t in enumerate(types)}
    n_classes = len(label_map)

    z = list(zip(xs, ys))
    ztr, zte = train_test_split(z, test_size=0.35, stratify=ys)
    Xtr, ytr = zip(*ztr)
    Xte, yte = zip(*zte)
    z = list(zip(Xtr, ytr))
    ztr, zval = train_test_split(z, test_size=0.15, stratify=ytr)
    Xtr, ytr = zip(*ztr)
    Xval, yval = zip(*zval)
    
    if scale:
        scaler = StandardScaler()
        scaler.fit(Xtr)
        Xtr = scaler.transform(Xtr)
        Xval = scaler.transform(Xval)
        Xte = scaler.transform(Xte)
        xs = scaler.transform(xs)
    
    if pca:
        scaler = PCA(n_components=5)
        scaler.fit(Xtr)
        Xtr = scaler.transform(Xtr)
        Xval = scaler.transform(Xval)
        Xte = scaler.transform(Xte)
        xs = scaler.transform(xs)

    train_dataset = ParameterDataset(Xtr, ytr, label_map)
    val_dataset = ParameterDataset(Xval, yval, label_map)
    test_dataset = ParameterDataset(Xte, yte, label_map)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(yte), shuffle=False)
    
    if linear:
        model = torch.nn.Sequential(
            torch.nn.Linear(len(xs[0]), n_classes)
        )
    else:
        model = torch.nn.Sequential(
            torch.nn.Linear(len(xs[0]), 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, n_classes)
        )

    types, counts = np.unique(ys, return_counts=True)
    ws = torch.tensor((1 / counts) / np.sum(1 / counts))

    criterion = torch.nn.CrossEntropyLoss(weight=ws)
    optim = torch.optim.Adam(model.parameters(), lr=0.005)
    epochs = 500

    train_accs = []
    train_f1s = []
    val_accs = []
    val_f1s = []

    for epoch in range(epochs):
        loss = 0
        for X, y, w in train_loader:
            X = X.to(torch.float32)
            y = y.to(torch.float32)
            y_pred = model(X)
            loss += criterion(y_pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        f_tr, acc_tr = summary(model, train_loader, n_classes)
        f_val, acc_val = summary(model, val_loader, n_classes)
        train_accs.append(acc_tr.item())
        train_f1s.append(f_tr.item())
        val_accs.append(acc_val.item())
        val_f1s.append(f_val.item())

        if (epoch + 1) % 10000 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item()}")

    #print_summary(model, train_loader, n_classes)
    f, acc = summary(model, test_loader, n_classes)
        
    return model, f.item(), acc.item()

In [ ]:
def kernel(x, a):
    ds = torch.tensor([1.0000, 0.6321, 0.3297, 0.1813, 0.0952, 0.0392, 0.0198, 0.0100])
    return torch.einsum("ij,j->i", torch.tensor(a).to(torch.float32), torch.pow(1 - ds, torch.tensor(x)))

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    
    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
with open("model/best_params.pickle", "rb") as f:
    all_params = pickle.load(f)

In [ ]:
instant = False
bin_size = 20
actv_bin_size = 20
#params = get_best_params_for_actv_bin_size(all_params, bin_size, actv_bin_size)
dataset = gen_dataset(all_params[(bin_size, actv_bin_size)])
xs, ys, cell_ids = get_dataset(dataset, instant)
ys = np.array(ys)

# !!!!!!!!!!!!!!!
types, counts = np.unique(ys, return_counts=True)
if instant:
    types = ["Htr3a", "Ndnf", "Vip", "Sst", "Pvalb", "Cux2", "Nr5a1", "Scnn1a", "Rorb", "Ntsr1"]#, "Rbp4"]
else:
    types = ["Htr3a", "Ndnf", "Vip", "Sst", "Pvalb", "Cux2", "Nr5a1", "Scnn1a", "Rorb", "Ntsr1", "Rbp4"]
inhibitory = ["Pvalb", "Sst", "Vip", "Htr3a", "Ndnf"]
# !!!!!!!!!!!!!!!

In [ ]:
wsss = []
for binary in [True, False]:
    print(f"{binary=}")
    if binary:
        labels = ["inhibitory" if y in inhibitory else "excitatory" for y in ys]
    else:
        labels = ys

    fs = []
    accs = []
    wss = []
    for i in range(30):
        model, f, acc = train_model(xs, labels, pca=False, scale=True, linear=True)
        fs.append(f)
        accs.append(acc)
        ws = model[0].weight.detach().norm(dim=0)
        ws = ws / torch.sum(ws)
        wss.append(ws)
    wsss.append(wss)
    
    print(f"({binary=}) f1 score: mean {np.mean(fs):.4f}, std {np.std(fs):.4f}")
    print(f"({binary=}) accuracy: mean {np.mean(accs):.4f}, std {np.std(accs):.4f}")
    print("------------------------")

In [ ]:
wsss[0] = torch.stack(wsss[0])
wsss[1] = torch.stack(wsss[1])

In [ ]:
wsss[0].std(axis=0).shape

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(7.5, 4.5), dpi=1000)
fig.text(-0.03, 0.5, 'relative importance', va='center', rotation='vertical')
ax1 = axs[0]
ax2 = axs[1]

if not instant:
    alpha = [f"$\\alpha_{i+1}$" for i in range(8)]
    beta = [f"$\\beta_{i+1}$" for i in range(8)]
    #actv = [f"$a_{i}$" for i in range(4 if actv_bin_size > 20 else 2)] + ["b", "max_current", "max_firing_rate"]
    actv = ["$c_0$", "$c_1$", "$\\gamma$"]
    labels = alpha + beta + actv
    ax1.bar(labels, wsss[0].mean(axis=0), yerr=wsss[0].std(axis=0), color=(["tab:green"]*8+["tab:purple"]*8+["lightblue"]*len(actv)))
    ax2.bar(labels, wsss[1].mean(axis=0), yerr=wsss[1].std(axis=0), color=(["tab:green"]*8+["tab:purple"]*8+["lightblue"]*len(actv)))
    ax1.set_xticks([])
    fig.tight_layout()
else:
    beta = [f"$\\beta_{i+1}$" for i in range(8)]
    #actv = [f"$a_{i}$" for i in range(4 if actv_bin_size > 20 else 2)] + ["b", "max_current", "max_firing_rate"]
    actv = ["$c_0$", "$c_1$", "$\\gamma$"]
    labels = beta + actv
    ax1.bar(labels, wsss[0].mean(axis=0), yerr=wsss[0].std(axis=0), color=(["tab:purple"]*8+["lightblue"]*len(actv)))
    ax2.bar(labels, wsss[1].mean(axis=0), yerr=wsss[1].std(axis=0), color=(["tab:purple"]*8+["lightblue"]*len(actv)))
    ax1.set_xticks([])
    fig.tight_layout()

In [ ]:
mapping = {t: i for i, t in enumerate(types)}
inv_mapping = {i: t for i, t in enumerate(types)}

xs_scaled = StandardScaler().fit_transform(xs)

n_clusters = 10 if instant else 11
clustering = AgglomerativeClustering(n_clusters=n_clusters, compute_distances=True, metric="euclidean", linkage="ward")
clustering.fit(xs_scaled)

plot_dendrogram(clustering, truncate_mode="lastp", p=n_clusters, leaf_rotation=60.,
            leaf_font_size=12.)
print(np.unique(clustering.labels_, return_counts=True))

id1 = []
id2 = []
mat = np.zeros((n_clusters, len(types)))
for i, y, x, cell_id in zip(clustering.labels_, ys, xs, cell_ids):
    #if i == 9:# and y == "Vip":
    #    print(cell_id, x)
    mat[i, mapping[y]] += 1

idx_map = [(j, i) for i, j in enumerate(sorted(range(mat.shape[0]), key=lambda i: mat[i, :] @ np.array([-1 if x < 5 else 1 for x in range(len(types))])))]
perm = np.zeros((n_clusters, n_clusters))
for i, j in idx_map:
    perm[j, i] = 1
mat = perm @ mat

In [ ]:
from matplotlib.collections import PatchCollection
from sklearn.preprocessing import normalize

N, M = mat.shape
ylabels = [f"Cluster {i+1}" for i in range(n_clusters)][::-1]
xlabels = [f"{inv_mapping[i]}" for i in range(len(types))]

x, y = np.meshgrid(np.arange(M), np.arange(N))
s = mat / mat.mean(axis=0)
c = np.zeros((N, M))
norm = mat / mat.max()
c[:, :5] = -2 - norm[:,:5]
c[:, 5:] = 2 + norm[:,5:]

fig, ax = plt.subplots(figsize=(7.5, 7.5 * N / M), dpi=1000)

#R = s/s.max()/2.2
R = normalize(s, norm='l2', axis=0) / 2.2
circles = [plt.Circle((j, i), radius=r, color='k', linewidth=10) for r, j, i in zip(R.flat, x.flat, y.flat)]
col = PatchCollection(circles, array=c.flatten(), cmap="bwr")
ax.add_collection(col)

ax.set(xticks=np.arange(M), yticks=np.arange(N),
       xticklabels=xlabels, yticklabels=ylabels)
ax.set_xticks(np.arange(M+1)-0.5, minor=True)
ax.set_yticks(np.arange(N+1)-0.5, minor=True)
ax.grid()
ax.set_axisbelow(True)
plt.xticks(rotation=90)
#fig.colorbar(col)

plt.show()

In [ ]:
import sklearn

for x in [xs_scaled]:
    clustering = AgglomerativeClustering(n_clusters=len(types), compute_distances=True, metric="euclidean", linkage="ward")
    clustering.fit(x);
    #print(sklearn.metrics.v_measure_score(ys, clustering.labels_))
    print(sklearn.metrics.adjusted_rand_score(ys, clustering.labels_))
    print(sklearn.metrics.v_measure_score(ys, clustering.labels_))

In [ ]:
dataset = gen_dataset(all_params[(bin_size, actv_bin_size)])
xs, ys, cell_ids = get_dataset(dataset, instant)
ys = np.array(ys)
types, counts = np.unique(ys, return_counts=True)
n_clusters = len(types)

In [ ]:
types = ["Htr3a", "Ndnf", "Vip", "Sst", "Pvalb", "Cux2", "Nr5a1", "Scnn1a", "Rorb", "Ntsr1", "Rbp4"]

In [ ]:
fig, ax = plt.subplots(len(types), 4, figsize=(12, 15), dpi=250)
for i, t in enumerate(types):
    a = xs[ys == t, :8]
    ax[i, 0].violinplot(a)
    ax[i, 0].xaxis.set_ticks(ticks=range(1, 9), labels=[str(j) for j in [0, 20, 50, 100, 200, 500, 1000, 2000]], rotation=90)
    
    zs = torch.linspace(0, 10, 500)
    X = torch.stack([kernel(z, a) for z in zs])
    x_mean = torch.mean(X, dim=1)
    x_std = torch.std(X, dim=1)
    
    
    ax[i, 1].plot(zs, x_mean)
    ax[i, 1].fill_between(zs, x_mean - x_std, x_mean + x_std, alpha=0.1, color="tab:blue")
    
    
    b = xs[ys == t, 8:16]
    ax[i, 2].violinplot(b)
    ax[i, 2].xaxis.set_ticks(ticks=range(1, 9), labels=[str(j) for j in [0, 20, 50, 100, 200, 500, 1000, 2000]], rotation=90)
    
    zs = torch.linspace(0, 50, 500)
    X = torch.stack([kernel(z, b) for z in zs])
    x_mean = torch.mean(X, dim=1)
    x_std = torch.std(X, dim=1)
    
    
    ax[i, 3].plot(zs, x_mean)
    ax[i, 3].fill_between(zs, x_mean - x_std, x_mean + x_std, alpha=0.1, color="tab:blue")
    
    ax[i, 0].set_ylabel(t)
    ax[i, 0].set_ylim([-0.5, 1])
    ax[i, 1].set_ylim([-0.2, 1.7])
    ax[i, 2].set_ylim([-2.5, 1.8])
    ax[i, 3].set_ylim([-5, 0.1])
    
    if i == 0:
        ax[i, 0].set_title("$\\alpha_i$")
        ax[i, 1].set_title("$k_I(t)$")
        ax[i, 2].set_title("$\\beta_i$")
        ax[i, 3].set_title("$k_f(t)$")
    
    if i < len(types)-1:
        for j in range(4):
            ax[i, j].xaxis.set_ticklabels([])
            
    if i == len(types) - 1:
        ax[i, 0].set_xlabel("$\\tau$")
        ax[i, 1].set_xlabel("$t$ (s)")
        ax[i, 2].set_xlabel("$\\tau$")
        ax[i, 3].set_xlabel("$t$ (s)")

fig.tight_layout()

In [ ]:
idx_map_inv = perm.argmax(axis=1)[::-1]

In [ ]:
clustering_labels = clustering.labels_
fig, ax = plt.subplots(n_clusters, 4, figsize=(12, 15), dpi=250)
for i in range(n_clusters):
    t = idx_map_inv[i]
    a = xs[clustering_labels == t, :8]
    ax[i, 0].violinplot(a)
    ax[i, 0].xaxis.set_ticks(ticks=range(1, 9), labels=[str(j) for j in [0, 20, 50, 100, 200, 500, 1000, 2000]], rotation=90)
    
    zs = torch.linspace(0, 10, 500)
    X = torch.stack([kernel(z, a) for z in zs])
    x_mean = torch.mean(X, dim=1)
    x_std = torch.std(X, dim=1)
    
    
    ax[i, 1].plot(zs, x_mean)
    ax[i, 1].fill_between(zs, x_mean - x_std, x_mean + x_std, alpha=0.1, color="tab:blue")
    
    
    b = xs[clustering_labels == t, 8:16]
    ax[i, 2].violinplot(b)
    ax[i, 2].xaxis.set_ticks(ticks=range(1, 9), labels=[str(j) for j in [0, 20, 50, 100, 200, 500, 1000, 2000]], rotation=90)
    
    zs = torch.linspace(0, 50, 500)
    X = torch.stack([kernel(z, b) for z in zs])
    x_mean = torch.mean(X, dim=1)
    x_std = torch.std(X, dim=1)
    
    
    ax[i, 3].plot(zs, x_mean)
    ax[i, 3].fill_between(zs, x_mean - x_std, x_mean + x_std, alpha=0.1, color="tab:blue")
    
    ax[i, 0].set_ylabel(f"Cluster {i+1}")
    ax[i, 0].set_ylim([-0.5, 0.7])
    ax[i, 1].set_ylim([-0.2, 2.3])
    ax[i, 2].set_ylim([-2.5, 1.8])
    ax[i, 3].set_ylim([-5, 0.1])
    
    if i == 0:
        ax[i, 0].set_title("$\\alpha_i$")
        ax[i, 1].set_title("$k_I(t)$")
        ax[i, 2].set_title("$\\beta_i$")
        ax[i, 3].set_title("$k_f(t)$")
    
    if i < n_clusters-1:
        for j in range(4):
            ax[i, j].xaxis.set_ticklabels([])
            
    if i == n_clusters - 1:
        ax[i, 0].set_xlabel("$\\tau$")
        ax[i, 1].set_xlabel("$t$ (s)")
        ax[i, 2].set_xlabel("$\\tau$")
        ax[i, 3].set_xlabel("$t$ (s)")

fig.tight_layout()

In [ ]:
clustering_labels = clustering.labels_
fig, ax = plt.subplots(4, 1, figsize=(8, 6), dpi=500)

for k, j in enumerate([0, 16, 17, 18]):
    d = []
    for t in types:
        d.append(xs[ys == t, j])

    #ax[k].violinplot(d, quantiles=[[0.25, 0.5, 0.75] for i in range(n_clusters)])
    ax[k].violinplot(d)
    if k < 3:
        ax[k].xaxis.set_ticks(ticks=list(range(1, len(types)+1)), labels=["" for i in range(len(types))])
    else:
        ax[k].xaxis.set_ticks(ticks=list(range(1, len(types)+1)), labels=types)

ax[0].set_ylabel("$\\alpha_1$")
ax[1].set_ylabel("$c_0$")
ax[2].set_ylabel("$c_1$")
ax[3].set_ylabel("$\\gamma$")
ax[3].set_xlabel("Cre-line")

ax[0].set_ylim([6.8, 9])
ax[1].set_ylim([-1.1, 0])
ax[2].set_ylim([-0.01, 0.16])
ax[3].set_ylim([0, 0.7])

fig.tight_layout()

In [ ]:
clustering_labels = clustering.labels_
fig, ax = plt.subplots(4, 1, figsize=(8, 6), dpi=500)

for k, j in enumerate([0, 16, 17, 18]):
    d = []
    for i in range(n_clusters):
        t = idx_map_inv[i]
        d.append(xs[clustering_labels == t, j])

    #ax[k].violinplot(d, quantiles=[[0.25, 0.5, 0.75] for i in range(n_clusters)])
    ax[k].violinplot(d)
    if k < 3:
        ax[k].xaxis.set_ticks(ticks=list(range(1, n_clusters+1)), labels=["" for i in range(n_clusters)])
    else:
        ax[k].xaxis.set_ticks(ticks=list(range(1, n_clusters+1)), labels=[f"{i+1}" for i in range(n_clusters)])

ax[0].set_ylabel("$\\alpha_1$")
ax[1].set_ylabel("$c_0$")
ax[2].set_ylabel("$c_1$")
ax[3].set_ylabel("$\\gamma$")
ax[3].set_xlabel("Cluster")

ax[0].set_ylim([6.5, 8.8])
ax[1].set_ylim([-1, -0.05])
ax[2].set_ylim([-0.01, 0.23])
ax[3].set_ylim([0, 0.7])

fig.tight_layout()